In [3]:
import numpy as np
from few.utils.modeselector import ModeSelector
from few.trajectory.inspiral import EMRIInspiral
from few.trajectory.ode.flux import KerrEccEqFlux
from few.amplitude.romannet import RomanAmplitude
from few.utils.ylm import GetYlms

use_gpu = False

if not use_gpu:
    
    import few
    
    #tune few configuration
    cfg_set = few.get_config_setter(reset=True)
    
    cfg_set.enable_backends("cpu")
    cfg_set.set_log_level("info");
    force_backend = 'cpu'
else:
    force_backend = 'gpu'
    pass #let the backend decide for itself.

In [8]:
traj = EMRIInspiral(func=KerrEccEqFlux)
amp = RomanAmplitude()

In [36]:
# parameters
m1 = 1e5
m2 = 1e1
a = 0.9
p0 = 10.0
e0 = 0.7
theta = np.pi / 3.0
phi = np.pi / 2.0

t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(m1, m2, a, p0, e0, 1.0)

teuk_modes = amp(a, p, e, x)
ylm_gen = GetYlms(include_minus_m=False)
ylms = ylm_gen(amp.unique_l, amp.unique_m, theta, phi).copy()[amp.inverse_lm]

mode_selector = ModeSelector(
    amp.l_arr_no_mask, amp.m_arr_no_mask, amp.n_arr_no_mask
)

mode_selection_threshold = 1e-1  # tolerance on mode contribution to total power

modeinds = [amp.l_arr, amp.m_arr, amp.n_arr]
(teuk_modes_in_selector, ylms_in, ls, ms, ns) = mode_selector(
    teuk_modes, ylms, modeinds, mode_selection_threshold=mode_selection_threshold
)

teuk_modes_in_selector.shape, ylms_in.shape

((82, 139), (278,))

In [37]:
# build a map: (l,m,n) -> index in teuk_modes
mode_map = {
    (int(l), int(m), int(n)): idx
    for idx, (l, m, n) in enumerate(zip(amp.l_arr, amp.m_arr, amp.n_arr))
}

# recover the indices for the cached (ls,ms,ns)
keep_inds = [mode_map[(int(l), int(m), int(n))] for l, m, n in zip(ls, ms, ns)]

# now just slice
teuk_modes_in_mapping = teuk_modes[:, keep_inds]
ylms_in_mapping = ylms[keep_inds]

In [38]:
(teuk_modes_in_selector == teuk_modes_in_mapping).all(), ylms_in.shape, ylms_in_mapping.shape

(np.True_, (278,), (139,))

In [35]:
keep_inds

[577,
 578,
 579,
 580,
 581,
 582,
 583,
 584,
 585,
 586,
 633,
 634,
 635,
 636,
 637,
 638,
 639,
 640,
 641,
 642,
 643,
 644,
 645,
 646,
 647,
 648,
 649,
 650,
 651,
 652,
 653,
 654,
 655,
 656,
 657,
 658,
 659,
 660,
 814,
 815,
 816,
 817,
 818,
 819,
 820,
 821,
 822,
 823,
 824,
 825,
 826,
 827,
 828,
 829,
 830,
 831,
 832,
 833,
 834,
 835,
 836,
 837,
 838,
 839,
 840,
 841,
 842,
 843,
 844,
 845,
 846,
 847,
 848,
 849,
 850,
 851,
 852,
 853,
 1067,
 1068,
 1069,
 1071,
 1072,
 1073,
 1075,
 1076,
 1077,
 1078,
 1080,
 1081,
 1082,
 1083,
 1084,
 1085,
 1086,
 1087,
 1088,
 1089,
 1093,
 1094,
 1095,
 1369,
 1371,
 1372,
 1373,
 1375,
 1376,
 1378,
 1379,
 1380,
 1388,
 1389,
 1390,
 1394,
 1395,
 1396,
 1736,
 1738,
 1742,
 1747,
 1748,
 1749,
 1750,
 1754,
 1758,
 1759,
 1768,
 2169,
 2170,
 2175,
 2176,
 2185,
 2186,
 2655,
 2660,
 2663,
 3203,
 3815,
 3816]